# Pars 1: URL to transcript, yt hash

In [1]:
from bs4 import BeautifulSoup
import numpy as np

import requests

url = "https://www.juznevesti.com/15-minuta/Marko-Nedeljkovic.sr.html"

req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")

In [2]:
text = ""
for i in soup.find_all("div"):
    current_classes_set = set(i.get("class", list()))
    if current_classes_set == {"bbredbolder", "mb20", "desc_holder"}:
        for j in i.find_all("p"):
            text += j.text + "\n"
video_div = soup.find(id="video_1")
video_link = video_div.contents[1].get("src")
yt_hash = video_link.split("/")[-1]
yt_hash


'Sk4JedgwClY'

In [3]:
video_div.contents

['\n',
 <iframe allowfullscreen="" frameborder="0" height="265" src="https://www.youtube.com/embed/Sk4JedgwClY" width="470"></iframe>,
 '\n']

# Pars 2: generating URLs.

In [4]:
found_urls = []
pagenumber = 2
for i in range(1, 29):
    url = f"https://www.juznevesti.com/Tagovi/Intervju-15-minuta.{pagenumber}.sr.html"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    divs = soup.find_all("div")
    for div in divs:
        current_class = div.get("class", [])
        if "t15min-list__item" in current_class or "t15min-list__featured" in current_class:
            found_urls.append([i  for i in div.find_all("a") if i.get("class", []) ==["db"]][0].get("href"))


# Fusion of the two parts

In [13]:
transcripts = list()
hashes = list()

for url in found_urls:
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")
    current_text = ""
    for i in soup.find_all("div"):
        current_classes_set = set(i.get("class", list()))
        if "desc_holder" in current_classes_set:
            current_text = "    ".join([j.text + "\n" for j in i.find_all("p")])
    video_div = soup.find(id="video_1")
    for element in video_div.contents:
        try:
            video_link = element.get("src", np.nan)
            break
        except KeyError:
            continue
    yt_hash = video_link.split("/")[-1]
    transcripts.append(current_text)
    hashes.append(yt_hash)

In [14]:
list(map(len, [hashes, transcripts, found_urls]))

[560, 560, 560]

In [15]:

import pandas as pd
df = pd.DataFrame(data={

    "url": found_urls,
    "yt_hash": hashes,
    "transcript": transcripts,
})

df

,url,yt_hash,transcript
0,https://www.juznevesti.com/Kultura/Price-na-di...,PFSd3XZTvgA,Zbirkom priča Blag reč književnica i novinarka...
1,https://www.juznevesti.com/Drushtvo/143-godine...,KfKWnocf7Ec,Mnogi sadašnji đaci nastaviće stopama nekada s...
2,https://www.juznevesti.com/Sport/Piksi-Juzna-p...,FxvpyOsw9eU,Morate da budete pobednik i da se ne predajete...
3,https://www.juznevesti.com/Kultura/Srdjan-Drag...,x2eOJEf8vUA,"Film Nebesa, čija je premijera održana i u Niš..."
4,https://www.juznevesti.com/Politika/Zelenovic-...,SQg4s0P64j8,Zeleno i levo je politika platforme koju okupl...
...,...,...,...
555,https://www.juznevesti.com/Drushtvo/Doktor-Dej...,YtiVyNOM05k,"Broj zaraženih raste, vakcinacija protiv koron..."
556,https://www.juznevesti.com/Drushtvo/Jova-Radov...,s5CMMcA1Mqc,"Šta nedostaje na današnjoj televiziji, zašto n..."
557,https://www.juznevesti.com/Drushtvo/Smrtnost-u...,MwzywjsTT0o,Upoređujući otvorene podatke istraživači Pravo...
558,https://www.juznevesti.com/Drushtvo/Price-o-lo...,_9zD8JdezlE,Koncentracioni logor koji se tokom Drugog svet...


In [16]:
df.isna().sum(axis=0)

url           0
yt_hash       0
transcript    0
dtype: int64

In [18]:
df.to_csv("006_crawling_juznevesti.csv", index=False)